# Assignment Q3

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import utils as ut

# Number of bodies in the simulation
nbodies = 5

# Time span for the simulation
tspan = np.arange(0, 1001, 1)

# Initial state vector (positions and orientations)
theta0 = np.array([0, 0, 0, 1, np.pi/4, np.pi/4, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0])
# Initial velocity vector (linear and angular velocities)
beta0 = np.zeros(14)
# Combine initial state and velocity into a single state vector
state0 = np.concatenate((theta0, beta0))

# Function to convert hinge angles to quaternions
def qfromtheta(theta,hingenum):
    q = np.zeros(4)

    if hingenum == 1 or hingenum == 4:
        q = theta

    elif hingenum == 2 or hingenum == 3:
        n = np.array([1, 0, 0])  # Axis of rotation (x-axes are colinear)
        q[:3] = np.sin(theta/2) * n
        q[3] = np.cos(theta/2)

    elif hingenum == 5:
        # NB: generazlied coordinates for body 4 consists of both rotation and translation, hence only the 4 first entries.
        q = theta[:4]
    return q

def ATBI(theta,beta,tau):
    # Initialize mass array for each body
    m = [None] * nbodies
    m[0] = 14520
    m[1] = 400
    m[2] = m[1]
    m[3] = m[1]
    m[4] = 294e3

    # Position of center of gravity relative to body frame in body frame:
    CGpos = [None] * nbodies
    CGpos[0] = np.array([0, 0, 0.7])
    CGpos[1] = np.array([0, 0, 2.5])
    CGpos[2] = CGpos[1]
    CGpos[3] = CGpos[1]
    CGpos[4] = np.array([0, 0, 0])

    # Position of outgoing hinge frame (body frame) relative to incoming hinge frame in body frame:
    Hpos = [None] * nbodies
    Hpos[0] = np.array([0, 0, 0])
    Hpos[1] = np.array([0, 0, 5])
    Hpos[2] = Hpos[1]
    Hpos[3] = Hpos[1]
    Hpos[4] = np.array([12, 4, 2])

    # 2nd order moment of inertia for each body in the CG frame:
    J = [None] * nbodies
    J[0] = np.diag([1.01, 1.51, 0.331]) * 1e4
    J[1] = np.diag([834, 834, 0.72])
    J[2] = J[1]
    J[3] = J[1]
    J[4] = np.diag([3.87, 29.04, 30.30]) * 1e6

    # Defining hinge-maps:
    H = [None] * nbodies
    H[0] = np.block([np.eye(3), np.zeros((3, 3))])
    H[1] = np.array([1, 0, 0, 0, 0, 0])
    H[2] = H[1]
    H[3] = H[0]
    H[4] = np.eye(6)

    # Rigid body transforms to move mass matrices from CG (where they are built) to body frame.
    O_phi_CG = [ut.rigidbodytransform(pos) for pos in CGpos]

    # Building spatial inertia matrices at body frame in body frame:
    M = [phi @ np.block([[J, np.zeros((3, 3))],
                         [np.zeros((3, 3)), J]]) @ phi.T for phi, J in zip(O_phi_CG, J)]
    
    # Building spatial vector q which holds quaternions for all bodies:
    q = [qfromtheta(t, j+1) for j, t in enumerate(theta)]

    # Preparing cells for values to store:
    V = [None] * nbodies
    agothic = [None] * nbodies
    bgothic = [None] * nbodies

    # Kinematics scatter:
    for k in reversed(range(nbodies)):
        # Rotation from child to parent:
        pRc = ut.spatialrotfromquat(q[k])
        # Rotation from parent to child:
        cRp = pRc.T

        # Rel. spatial vel. across hinge:
        Delta_V = H[k] * beta[k]

        # Spatial velocities (using if statement to enforce V(0)=0):
        if k == nbodies - 1:
            V[k] = Delta_V
        else:
            phi = ut.rigidbodytransform(Hpos[k+1])
            V[k] = cRp @ phi.T @ V[k+1] + Delta_V

        # Coriolis term (const. joint map and pure rotation):
        if k == nbodies - 1:
            agothic[k] = ut.skewfromspatialvec(V[k]) @ Delta_V - ut.skewbarfromspatialvec(Delta_V) @ Delta_V
        else:
            agothic[k] = ut.skewfromspatialvec(V[k]) @ Delta_V

        # Gyroscopic term:
        bgothic[k] = ut.skewbarfromspatialvec(V[k]) @ M[k] @ V[k]

    # Preparing cells for values to store:
    Pplus = [None] * nbodies
    G = [None] * nbodies
    nu = [None] * nbodies
    varepsplus = [None] * nbodies
    alpha = [None] * nbodies
    gamma = [None] * nbodies

    # ATBI gather (using if statement to enforce Pplus(0)=0, varepsplus(0)=0):
    for k in range(nbodies):
        if k == 1:
            P = M[k]
            D = H[k] @ P @ H[k].T
            G[k] = P @ H[k].T @ np.linalg.inv(D)
            taubar = np.eye(6) - G[k] @ H[k]
            Pplus[k] = taubar * P
            vareps = P @ agothic[k] + bgothic[k]
            eps = tau[k] - H[k] @ vareps
            nu[k] = np.linalg.inv(D) @ eps
            varepsplus[k] = vareps + G[k] @ eps
        else:
            # Rotation from parent to child:
            pRc = ut.spatialrotfromquat(q[k-1])
            # Rotation from child to parent:
            cRp = pRc.T
            # Rigid body transform:
            phi = ut.rigidbodytransform(Hpos[k])

            P = phi @ pRc @ Pplus[k-1] @ cRp @ phi.T + M[k]
            D = H[k] @ P @ H[k].T
            G[k] = P @ H[k].T @ np.linalg.inv(D)
            taubar = np.eye(6) - G[k] @ H[k]
            Pplus[k] = taubar @ P
            vareps = phi @ pRc @ varepsplus[k-1] + P @ agothic[k] + bgothic[k]
            eps = tau[k] - H[k] @ vareps
            nu[k] = np.linalg.inv(D) @ eps
            varepsplus[k] = vareps + G[k] @ eps

    # ATBI scatter (using if statement to enforce alpha(n+1)=0):
    for k in reversed(range(nbodies)):
        if k == nbodies - 1:
            alphaplus = np.zeros(6)
            gamma[k] = nu[k] - G[k].T @ alphaplus
            alpha[k] = alphaplus + H[k].T @ gamma[k] + agothic[k]
        else:
            # Rotation from child to parent:
            pRc = ut.spatialrotfromquat(q[k])
            # Rotation from parent to child:
            cRp = pRc.T
            # Rigid body transform:
            phi = ut.rigidbodytransform(Hpos[k+1])

            alphaplus = cRp @ phi.T @ alpha[k+1]
            gamma[k] = nu[k] - G[k].T @ alphaplus
            alpha[k] = alphaplus + H[k].T @ gamma[k] + agothic[k]
    return alpha, gamma

# ODE FUN HERE:
def 







Processing body index: 4
Processing body index: 3
Processing body index: 2
Processing body index: 1
Processing body index: 0
